In [ ]:
!pip install langchain langchain-community gigachat unstructured "unstructured[pdf]" langchain-huggingface numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 26.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 22.2 MB/s eta 0:00:00


In [ ]:
!pip install -U langchain-community

In [ ]:
!pip install rank_bm25

In [ ]:
import os
import numpy as np
from typing import List
from pydantic import BaseModel, Field
from getpass import getpass

# --- Установка и импорт необходимых библиотек ---
# !pip install langchain langchain-community gigachat unstructured "unstructured[pdf]" langchain-huggingface faiss numpy

# --- Импорт необходимых компонентов ---
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA, LLMChain
from langchain_community.chat_models import GigaChat
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.retrievers.multi_query import MultiQueryRetriever

# --- Настройка LLM ---
if "GIGACHAT_CREDENTIALS" not in os.environ:
    os.environ["GIGACHAT_CREDENTIALS"] = getpass("Введите ваши credentials GigaChat: ")

# Инициализация LLM
llm = GigaChat(verify_ssl_certs=False, scope="GIGACHAT_API_PERS")

# --- 1. Загрузка документов ---
# Укажите путь к вашей директории с документами
docs_path = "/content/docs"  # Обновите на ваш путь
loader = DirectoryLoader(docs_path, glob="**/*.pdf")
docs = loader.load()

print(f"Загружено {len(docs)} документов")

# --- 2. Улучшенное разделение на фрагменты ---
# Используем разделители, которые могут указывать на границы услуг
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],
    keep_separator=False
)
chunks = splitter.split_documents(docs)

print(f"Создано {len(chunks)} фрагментов текста")

# --- 3. Улучшенные эмбеддинги с мультиязычной моделью ---
emb = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large",  # Лучшая мультиязычная модель
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

# Создаем векторную базу данных
vectordb = FAISS.from_documents(chunks, emb)

Введите ваши credentials GigaChat: ··········


<ipython-input-4-c5a3bc773d2d>:27: LangChainDeprecationWarning: The class `GigaChat` was deprecated in LangChain 0.3.5 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-gigachat package and should be used instead. To use it run `pip install -U :class:`~langchain-gigachat` and import as `from :class:`~langchain_gigachat import GigaChat``.
  llm = GigaChat(verify_ssl_certs=False, scope="GIGACHAT_API_PERS")


Загружено 1 документов
Создано 48 фрагментов текста


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [ ]:

# --- 4. Создание гибридного поиска ---
# BM25 для поиска по ключевым словам
bm25_retriever = BM25Retriever.from_documents(chunks, k=15)

# Гибридный поиск, комбинирующий семантический и поиск по ключевым словам
hybrid_retriever = EnsembleRetriever(
    retrievers=[
        vectordb.as_retriever(search_kwargs={"k": 10}),
        bm25_retriever
    ],
    weights=[0.7, 0.3]
)

# --- 5. Функция для ранжирования документов с помощью LLM ---
def rerank_with_llm(user_query, retrieved_docs, top_k=5):
    """Ранжирование документов с помощью LLM для более точной оценки релевантности"""

    # Шаблон для оценки релевантности
    scoring_template = """
    На шкале от 0 до 10, оцени насколько следующее описание услуги соответствует запросу пользователя.
    Запрос пользователя: {query}

    Описание услуги: {service}

    Оценка (только число от 0 до 10):
    """

    scoring_prompt = PromptTemplate(
        template=scoring_template,
        input_variables=["query", "service"]
    )

    # Оцениваем каждый документ
    scored_docs = []
    for doc in retrieved_docs:
        score_chain = LLMChain(llm=llm, prompt=scoring_prompt)
        try:
            score = float(score_chain.run(query=user_query, service=doc.page_content).strip())
            scored_docs.append((doc, score))
        except:
            scored_docs.append((doc, 0))

    # Сортируем документы по оценке и возвращаем лучшие
    scored_docs.sort(key=lambda x: x[1], reverse=True)
    return [doc for doc, score in scored_docs[:top_k]]

# --- 6. Структуры данных для результатов ---
class ServiceRecommendation(BaseModel):
    service_id: str = Field(description="ID услуги")
    relevance_score: int = Field(description="Оценка релевантности от 1 до 10")
    reason: str = Field(description="Краткое объяснение, почему эта услуга подходит")

class ServiceRecommendations(BaseModel):
    recommendations: List[ServiceRecommendation] = Field(description="Одна наиболее подходящая услуга, для которой указана оценка релевантонсти и описание самой услуги")

# Создаем парсер для структурированного вывода
parser = PydanticOutputParser(pydantic_object=ServiceRecommendations)

# --- 7. Цепочка для генерации вариаций запроса ---
query_variations_template = """
Пользователь описывает свою проблему в сфере образования и ищет наиболее подходящую услугу. По запросу пользователя тебе нужно выдать категорию, к которой вероятнее всего относится услуга, которую он ищет, по которой будет проводиться предварительный отбор кандидатов в подходящие услуги из общей базы.
Исходный запрос: {query}
Список возможных категорий, из которых тебе необохдимо выбрать: биология, высшая математика, физика, философия, химия, экономика, программирование, история, социология, архитекрура ЭВМ, литература

"""

query_variations_prompt = PromptTemplate(
    template=query_variations_template,
    input_variables=["query"]
)

variations_chain = LLMChain(llm=llm, prompt=query_variations_prompt)

# --- 8. Цепочка для ранжирования результатов ---
ranking_prompt = PromptTemplate(
    input_variables=["original_query", "retrieved_services"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
    template="""
    Ты эксперт по подбору услуг на основе запроса пользователя.

    Оригинальный запрос пользователя: {original_query}

    Вот список потенциально релевантных услуг:
    {retrieved_services}

    Ты уже выяснил специализацию. Предоставь мне услуги только по данной специ
    Выведи сначала специализацию и предметную и научную области, которые ты выявил
    Выбери ТОЛЬКО две наиболее подходящих услуг для запроса пользователя, подходящих по специализации запросу. Если услуга пользователя хоть немного не подходит под описание, например, если область изучения хотя бы чуть-чуть не совпадает с областью изучения пользователя, то ни в коем случае не добавляй ее в список.
    Оцени услугу по шкале от 1 до 10, где 10 - идеально подходит.
    Для услуги напиши краткое объяснение, почему она подходит.
    Не смотри на оценки ранжирования. Не обращай внимание на теги. Не обращай внимание на время и дату. Обращай внимание именно на текст и описание. Например, если пользователю нужна помощь с математикой, отдай предпочтение услуги репетитора по математике, а не другим наукам
    Выпиши ID подходящих людей, способных справиться с задачей и их специализацию. Внимательно следи за тем, чтобы ID и услуга действительно совпадали, когда будешь выдавать ответ.
    Если ты сомневаешься или не можешь найти предметную область для запроса пользователя. Лучше напиши, что ты не можешь найти нужные услуги. Тебя нельзя рекомендовать услуги, чьи научная и предмтная область отличается от предметной и научной области запроса.
    Также ни в коем случае не предлагай услуги, которые только могут быть подезными, но по факту не являются таковыми.
    НИ В КОЕМ СЛУЧАЕ НЕ ПРЕДЛАГАЙ СМЕЖНЫЕ ОБЛАСТИ! НУЖНА ТОЛЬКО ОБЛАСТЬ, КОТОРУЮ ТЫ ВЫЯСНИЛ ИЗ ЗАПРОСА!!!
    {format_instructions}
    """
)

ranking_chain = LLMChain(
    llm=llm,
    prompt=ranking_prompt
)

# --- 9. Улучшенная функция поиска услуг ---
def enhanced_service_search(user_query):
    """Расширенный поиск с использованием множественных запросов и ре-ранжированием"""
    print("🔍 Начинаю поиск подходящих услуг...")

    # 1. Генерируем вариации запроса
    variations_result = variations_chain.run(query=user_query)
    query_variations = [q.strip() for q in variations_result.strip().split('\n') if q.strip()]
    query_variations.append(user_query)  # Добавляем исходный запрос

    print(f"📝 Сгенерированы вариации запроса: {query_variations}")

    # 2. Получаем документы по всем вариациям запроса
    all_docs = []
    #for query in query_variations:
    docs = hybrid_retriever.get_relevant_documents(query_variations[0])
    all_docs.extend(docs)

    # 3. Удаляем дубликаты
    unique_docs = []
    seen_content = set()
    for doc in all_docs:
        if doc.page_content not in seen_content:
            seen_content.add(doc.page_content)
            unique_docs.append(doc)

    print(f"📚 Найдено {len(unique_docs)} уникальных документов")

    # 4. Ре-ранжируем с помощью LLM
    reranked_docs = rerank_with_llm(user_query, unique_docs, top_k=8)
    print(reranked_docs)
    # 5. Форматируем контекст для финального ранжирования
    services_context = "\n\n".join([f"ID: {i+1}\n" + doc.page_content for i, doc in enumerate(reranked_docs)])

    # 6. Запускаем финальное ранжирование
    try:
        ranking_result = ranking_chain.run(
            original_query=user_query,
            retrieved_services=services_context
        )
        parsed_result = parser.parse(ranking_result)
        return parsed_result
    except Exception as e:
        print(f"⚠️ Ошибка при обработке результатов: {e}")
        try:
            # Попытка извлечь полезную информацию даже при ошибке парсинга
            return ranking_result
        except:
            return "Не удалось найти подходящие услуги для вашего запроса."

# --- 10. Интерфейс пользователя ---
def get_service_recommendations():
    """Основная функция для взаимодействия с пользователем"""
    print("=" * 80)
    print("📋 Система рекомендации услуг")
    print("=" * 80)

    user_query = input("🙋 Опишите вашу проблему: ")
    print("⏳ Идет подбор подходящих услуг, пожалуйста подождите...")

    results = enhanced_service_search(user_query)

    if isinstance(results, ServiceRecommendations):
        print("\n✅ Рекомендуемые услуги:")
        for i, rec in enumerate(results.recommendations):
            print(f"{i+1}. 🆔 ID услуги: {rec.service_id}")
            print(f"   ⭐ Релевантность: {rec.relevance_score}/10")
            print(f"   💬 Почему подходит: {rec.reason}\n")
    else:
        print(f"📝 Результаты поиска:\n{results}")

    print("=" * 80)

# Запуск системы
get_service_recommendations()

📋 Система рекомендации услуг
🙋 Опишите вашу проблему: Мне нужна помощь с дз по ассемблеру
⏳ Идет подбор подходящих услуг, пожалуйста подождите...
🔍 Начинаю поиск подходящих услуг...
📝 Сгенерированы вариации запроса: ['Программирование', 'Мне нужна помощь с дз по ассемблеру']
📚 Найдено 22 уникальных документов
[Document(id='aad94a9a-66fd-4f37-a88e-2bc44858120d', metadata={'source': '/content/docs/sample_document.pdf'}, page_content='category: естественные науки\n\nprice: 1500\n\ncurrency: RUB\n\nprovider_id: 3063\n\ncreated_at: 2024-10-13T15:52:14Z\n\ntags: химия, лабораторные, органика\n\navailable: True\n\nid: 64\n\ntitle: Решение задач по программированию\n\ndescription: Помощь с заданиями по Python, C++, Java. Объясняю не только решение, но и сам\n\nпринцип. Работаю в ИТ-компании, стаж более 7 лет. Провожу онлайн-занятия.\n\ncategory: информатика\n\nprice: 1200\n\ncurrency: RUB\n\nprovider_id: 3064\n\ncreated_at: 2024-08-15T22:50:27Z\n\ntags: программирование, python, java, C++\n\na